MinerU - PDF Test

- for installization : https://github.com/opendatalab/MinerU/blob/master/docs/README_Ubuntu_CUDA_Acceleration_en_US.md
- usage for python API : https://mineru.readthedocs.io/en/latest/user_guide/usage/api.html



In [3]:
import os
from magic_pdf.data.data_reader_writer import FileBasedDataWriter, FileBasedDataReader
from magic_pdf.data.dataset import PymuDocDataset
from magic_pdf.model.doc_analyze_by_custom_model import doc_analyze
from magic_pdf.config.enums import SupportedPdfParseMethod

##### Example, convert pdf

In [5]:


input_dir = '/home/xiong/data/Fund/pdf_parse'
output_dir = os.path.join(input_dir, 'temp','output')
pdf_file_path = os.path.join(input_dir, 'temp','111_2023_0_table1.pdf')  # replace with the real pdf path

In [6]:
## specify output dir
name_without_suffix = os.path.splitext(os.path.basename(pdf_file_path))[0]
pdf_output_dir = os.path.join(output_dir, name_without_suffix)
local_image_dir = os.path.join(pdf_output_dir, "images")
local_md_dir = pdf_output_dir
os.makedirs(local_image_dir, exist_ok=True)
os.makedirs(local_md_dir, exist_ok=True)

In [7]:
### run pdf processing
# read bytes
image_writer = FileBasedDataWriter(local_image_dir)
md_writer = FileBasedDataWriter(local_md_dir)
# Readers
reader = FileBasedDataReader("")
pdf_bytes = reader.read(pdf_file_path)
# Processing
ds = PymuDocDataset(pdf_bytes)
## inference
if ds.classify() == SupportedPdfParseMethod.OCR:
    infer_result = ds.apply(doc_analyze, ocr=True)
    pipe_result = infer_result.pipe_ocr_mode(image_writer)
else:
    infer_result = ds.apply(doc_analyze, ocr=False)
    pipe_result = infer_result.pipe_txt_mode(image_writer)

2025-04-13 00:54:13.673 | INFO     | magic_pdf.data.dataset:__init__:157 - lang: None


2025-04-13 00:54:13.830 | INFO     | magic_pdf.libs.pdf_check:detect_invalid_chars:67 - cid_count: 0, text_len: 3258, cid_chars_radio: 0.0


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/home/xiong/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


/home/xiong/miniconda3/envs/llm/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [38]:
from IPython.display import HTML, display

In [39]:
### get model inference result
model_inference_result = infer_result.get_infer_res()
md_content = pipe_result.get_markdown(local_image_dir)
content_list_content = pipe_result.get_content_list(local_image_dir)

In [40]:
display(HTML(content_list_content[0]['table_body']))

In [41]:
class PDFConverter:
    def __init__(self, input_dir, output_dir):
        self.input_dir = input_dir
        self.output_dir = output_dir

    def process_pdf(self, pdf_file_path,draw_pdf=False):
        # Create a unique directory for each PDF
        name_without_suffix = os.path.splitext(os.path.basename(pdf_file_path))[0]
        pdf_output_dir = os.path.join(self.output_dir, name_without_suffix)
        local_image_dir = os.path.join(pdf_output_dir, "images")
        local_md_dir = pdf_output_dir
        os.makedirs(local_image_dir, exist_ok=True)
        os.makedirs(local_md_dir, exist_ok=True)
        # read bytes
        image_writer = FileBasedDataWriter(local_image_dir)
        md_writer = FileBasedDataWriter(local_md_dir)
        # Readers
        reader = FileBasedDataReader("")
        pdf_bytes = reader.read(pdf_file_path)
        # Processing
        ds = PymuDocDataset(pdf_bytes)

        ## inference
        if ds.classify() == SupportedPdfParseMethod.OCR:
            infer_result = ds.apply(doc_analyze, ocr=True)
            pipe_result = infer_result.pipe_ocr_mode(image_writer)
        else:
            infer_result = ds.apply(doc_analyze, ocr=False)
            pipe_result = infer_result.pipe_txt_mode(image_writer)

        # Output
        if draw_pdf:
            infer_result.draw_model(os.path.join(local_md_dir, f"{name_without_suffix}_model.pdf"))
            pipe_result.draw_layout(os.path.join(local_md_dir, f"{name_without_suffix}_layout.pdf"))
            pipe_result.draw_span(os.path.join(local_md_dir, f"{name_without_suffix}_spans.pdf"))

        md_content = pipe_result.get_markdown(local_image_dir)
        pipe_result.dump_md(md_writer, f"{name_without_suffix}.md", local_image_dir)
        pipe_result.dump_content_list(md_writer, f"{name_without_suffix}_content_list.json", local_image_dir)
        pipe_result.dump_middle_json(md_writer, f"{name_without_suffix}_middle.json")

    def process_all_pdfs(self):
        for filename in os.listdir(self.input_dir):
            if filename.endswith(".pdf"):
                pdf_file_path = os.path.join(self.input_dir, filename)
                self.process_pdf(pdf_file_path)



In [43]:
input_folder = '/ephemeral/home/xiong/data/Fund/pdf_parse'
output_folder = os.path.join(input_dir, 'output')
pdf_file_path = os.path.join(input_dir, '111_2023_0_table1.pdf')  # replace with the real pdf path
converter = PDFConverter(input_folder, output_folder)
converter.process_all_pdfs()

2025-03-02 21:31:20.995 | INFO     | magic_pdf.data.dataset:__init__:156 - lang: None
2025-03-02 21:31:21.057 | INFO     | magic_pdf.libs.pdf_check:detect_invalid_chars:67 - cid_count: 0, text_len: 3258, cid_chars_radio: 0.0
2025-03-02 21:31:22.324 | INFO     | magic_pdf.model.pdf_extract_kit:__call__:210 - layout detection time: 1.25
2025-03-02 21:31:25.956 | INFO     | magic_pdf.model.pdf_extract_kit:__call__:216 - mfd time: 3.63
2025-03-02 21:31:31.807 | INFO     | magic_pdf.model.pdf_extract_kit:__call__:223 - formula nums: 13, mfr time: 5.85
2025-03-02 21:31:31.990 | INFO     | magic_pdf.model.pdf_extract_kit:__call__:257 - det time: 0.18
2025-03-02 21:31:40.271 | INFO     | magic_pdf.model.pdf_extract_kit:__call__:297 - table time: 8.28
2025-03-02 21:31:40.272 | INFO     | magic_pdf.model.doc_analyze_by_custom_model:doc_analyze:228 - -----page_id : 0, page total time: 19.2-----
2025-03-02 21:31:40.553 | INFO     | magic_pdf.model.doc_analyze_by_custom_model:doc_analyze:239 - gc t

#### Print out the markdown content 

In [46]:
# Get the markdown file path from the processed PDF
name_without_suffix = os.path.splitext(os.path.basename(pdf_file_path))[0]
md_file_path = os.path.join(output_folder,name_without_suffix, name_without_suffix + '.md')

# Read and display markdown content if file exists
if os.path.exists(md_file_path):
    # Read markdown content
    with open(md_file_path, 'r', encoding='utf-8') as f:
        md_content = f.read()
    # Display markdown content inline using IPython
    from IPython.display import display, Markdown
    display(Markdown(md_content))
else:
    print(f"Markdown file not found at: {md_file_path}")


<html><body><table><tr><td colspan="10">Table 1.United States: Selected Economic Indicators (Percentage change from previous period, unless otherwise indicated)</td></tr><tr><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>Projections</td><td></td><td></td><td></td></tr><tr><td></td><td>2019</td><td>2020</td><td>2021</td><td>2022</td><td>2023</td><td>2024</td><td>2025</td><td>2026 2027</td><td>2028</td><td></td></tr><tr><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td></tr><tr><td>National Production and Income</td><td>2.3</td><td></td><td>5.9</td><td></td><td></td><td></td><td></td><td></td><td>2.1</td><td>2.1</td></tr><tr><td>Real GDP</td><td>2.6</td><td>-2.8 -1.5</td><td></td><td>2.1</td><td>1.7</td><td>1.0</td><td>1.8</td><td>2.1</td><td>2.1</td><td>2.1</td></tr><tr><td>Real GDP (q4/q4) Net exports 1/</td><td>-0.1</td><td>-0.3</td><td>5.7 -1.2</td><td>0.9 -0.4</td><td>1.2 0.5</td><td>1.1</td><td>2.0 0.0</td><td>2.1 0.0</td><td>0.0</td><td>0.0</td></tr><tr><td>Total domesticdemand</td><td>2.3</td><td>-2.4</td><td>7.0</td><td>2.4</td><td>1.1</td><td>0.0</td><td>1.7</td><td>2.0</td><td>2.1</td><td></td></tr><tr><td>Final domestic demand</td><td>2.3</td><td>-1.9</td><td>6.7</td><td>1.7</td><td>1.6</td><td>1.0 1.0</td><td>1.7</td><td>2.1</td><td>2.1</td><td>2.1 2.1</td></tr><tr><td>Privatefinalconsumption</td><td>2.0</td><td>-3.0</td><td>8.3</td><td>2.7</td><td>2.0</td><td>0.8</td><td>1.4</td><td>1.7</td><td>2.0</td><td>2.0</td></tr><tr><td></td><td>3.4</td><td>2.2</td><td>1.3</td><td>-0.2</td><td>3.0</td><td>1.4</td><td>1.3</td><td>1.3</td><td>1.3</td><td>1.3</td></tr><tr><td>Public consumption expenditure</td><td>2.6</td><td>-1.2</td><td>5.7</td><td>-0.5</td><td>-0.8</td><td>1.3</td><td>3.2</td><td>3.7</td><td>3.2</td><td>3.1</td></tr><tr><td>Grossfixeddomesticinvestment</td><td>2.5</td><td>-2.3</td><td>7.4</td><td>-0.2</td><td>-1.9</td><td>0.7</td><td>3.0</td><td>3.7</td><td>3.7</td><td>3.8</td></tr><tr><td>Privatefixedinvestment</td><td>3.1</td><td>3.9</td><td>-2.3</td><td>-2.1</td><td>4.9</td><td>4.1</td><td>3.9</td><td>3.5</td><td>0.5</td><td>0.0</td></tr><tr><td>Publicfixedinvestment</td><td>0.0</td><td>-0.5</td><td>0.2</td><td>0.7</td><td>-0.5</td><td>0.0</td><td>0.0</td><td>0.0</td><td>0.0</td><td>0.0</td></tr><tr><td>Change in private inventories 1/</td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td></tr><tr><td>Nominal GDP</td><td>4.1</td><td>-1.5</td><td>10.7</td><td>9.2</td><td>6.0</td><td>4.1</td><td>4.3</td><td>4.2</td><td>4.1</td><td>4.1</td></tr><tr><td>Personal saving rate (%of disposableincome)</td><td>8.8</td><td>16.8 17.3</td><td>11.9 17.6</td><td>3.5 18.2</td><td>4.1 17.2</td><td>4.0 17.1</td><td>4.9 17.3</td><td>4.9 17.5</td><td>4.9 17.8</td><td>5.4 18.0</td></tr><tr><td>Private investment rate (% of GDP)</td><td>17.8</td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td></tr><tr><td>Unemployment and Potential Output</td><td></td><td></td><td></td><td>3.6</td><td>3.7</td><td>4.2</td><td>4.3</td><td>4.0</td><td></td><td>4.0</td></tr><tr><td>Unemploymentrate Labor force participation rate</td><td>3.7 63.1</td><td>8.1 61.7</td><td>61.7</td><td>62.2</td><td>62.5</td><td>62.5</td><td>62.4</td><td>62.2</td><td>4.0 62.2</td><td>62.2</td></tr><tr><td>Potential GDP</td><td>1.6</td><td>0.4</td><td></td><td>2.2</td><td>2.1</td><td></td><td>2.0</td><td>2.0</td><td>2.0</td><td></td></tr><tr><td>Output gap (% of potential GDP)</td><td>0.7</td><td>-2.5</td><td>1.8 1.5</td><td>1.4</td><td>0.9</td><td>2.0 0.0</td><td>-0.2</td><td>-0.2</td><td>-0.1</td><td>2.0 0.0</td></tr><tr><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td></tr><tr><td>Inflation</td><td></td><td>1.2</td><td>6.8</td><td>7.1</td><td>3.6</td><td>2.5</td><td>2.4</td><td>2.1</td><td></td><td>2.2</td></tr><tr><td>CPl inflation (q4/q4) Core CPl Inflation (q4/q4)</td><td>2.0 2.3</td><td>1.6</td><td></td><td>6.0</td><td>4.2</td><td>2.9</td><td>2.7</td><td>2.3</td><td>2.1 2.3</td><td>2.3</td></tr><tr><td>PCE Inflation (q4/q4)</td><td>1.4</td><td>1.1</td><td>5.0 5.7</td><td>5.7</td><td>3.8</td><td>2.6</td><td>2.3</td><td>1.9</td><td>1.9</td><td>2.0</td></tr><tr><td>Core PCE Inflation (q4/q4)</td><td>1.6</td><td>1.4</td><td>4.7</td><td>4.8</td><td>4.1</td><td>2.8</td><td>2.5</td><td>2.0</td><td>2.0</td><td>2.0</td></tr><tr><td>GDP deflator</td><td>1.8</td><td>1.3</td><td>4.5</td><td>7.0</td><td>4.3</td><td>3.0</td><td>2.5</td><td>2.1</td><td>1.9</td><td>1.9</td></tr><tr><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td></tr><tr><td>Government Finances</td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>-6.2</td><td></td><td>-6.4</td></tr><tr><td>Federal balance (% of GDP) 2/</td><td>-4.7</td><td>-14.9</td><td>-12.3</td><td>-5.5</td><td>-5.6</td><td>-5.7</td><td>-6.4</td><td></td><td>-5.9</td><td>108.3</td></tr><tr><td>Federal debt held by the public (% of GDP)</td><td>79.4</td><td>99.8</td><td>98.4</td><td>97.0</td><td>96.6</td><td>98.4</td><td>101.2</td><td>103.6</td><td>105.8</td><td></td></tr><tr><td>General governmentbudgetbalance(%of GDP)</td><td>-5.7</td><td>-14.0</td><td>-11.6 126.4</td><td>-3.7</td><td>-6.7</td><td>-7.0</td><td>-7.3</td><td>-7.1</td><td>-6.9</td><td>-7.0</td></tr><tr><td>Generalgovernmentgross debt (%of GDP)</td><td>108.7</td><td>133.5</td><td></td><td>121.4</td><td>121.8</td><td>124.6</td><td>127.5</td><td>130.1</td><td>132.5</td><td>134.9</td></tr><tr><td>Interest Rates (percent; period average)</td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td></td><td>2.4</td></tr><tr><td>Fed funds rate</td><td>2.2</td><td>0.4</td><td>0.1</td><td>1.7</td><td>5.1</td><td>5.3</td><td>4.2</td><td>3.2</td><td>2.4</td><td></td></tr><tr><td>Three-month Treasury bill rate Ten-year government bond rate</td><td>2.1 2.1</td><td>0.4 0.9</td><td>0.0</td><td>2.1 3.0</td><td>5.2 3.8</td><td>5.3 3.8</td><td>4.2 3.6</td><td>3.2 3.4</td><td>2.4 3.4</td><td>2.4 3.4</td></tr><tr><td>Balance of Payments</td><td></td><td></td><td>1.4</td><td></td><td></td><td></td><td></td><td></td><td></td></table></body></html>

Sources: BEA; BLS; FRB; Haver Analytics; and IMF staff estimates. 1/ Contribution to real GDP growth, percentage points. 2/ Includes staff's adjustments for one-off items, including costs of financial sector support.  